Nama : Sakinah Yunus

NIM  : H071191046



***DEEP LEARNING COMPUTATION ***

In [15]:
import numpy as np
import tensorflow as tf

**5.1. Layers and Blocks**

In [16]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.12686032,  0.03298973, -0.36464423,  0.6374717 , -0.04941432,
         0.08728278, -0.18474706,  0.13569272, -0.33187324, -0.1337718 ],
       [ 0.09682041,  0.04847944, -0.4100217 ,  0.42526528, -0.14503987,
         0.16230184, -0.06455754,  0.2368817 , -0.3249419 , -0.07545272]],
      dtype=float32)>

**5.1.1. A Custom Block**

In [17]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)
    def call(self, X):
        return self.out(self.hidden((X)))

In [18]:
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.00222629,  0.1426515 ,  0.02307511, -0.01004117,  0.04867152,
        -0.20869921,  0.22447386, -0.18684225, -0.00392289,  0.19304514],
       [ 0.0591289 , -0.00253747, -0.0841844 , -0.08366194,  0.0227528 ,
         0.00949792,  0.16004632, -0.10908736,  0.08352487,  0.14336213]],
      dtype=float32)>

**5.1.2. The Sequential Block**

In [20]:
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        for block in args:
            self.modules.append(block)

    def call(self, X):
        for module in self.modules:
            X = module(X)
        return X

In [21]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-1.56814121e-02,  5.99248856e-02, -2.13919997e-01,
         2.71268189e-04, -1.33849680e-01,  1.09549545e-01,
         1.63846001e-01,  1.08523965e-02,  3.50816101e-02,
         3.05658042e-01],
       [-4.37335409e-02, -3.06530446e-02, -3.51165056e-01,
         1.47224903e-01, -1.12702549e-02,  3.20377886e-01,
         1.03378817e-01,  8.08011666e-02,  1.19670115e-01,
         1.23644561e-01]], dtype=float32)>

**5.1.3. Executing Code in The Forward Propagation Function**


In [22]:
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.rand_weight = tf.constant(tf.random.uniform((20, 20)))
        self.dense = tf.keras.layers.Dense(20, activation=tf.nn.relu)

    def call(self, inputs):
        X = self.flatten(inputs)
        X = tf.nn.relu(tf.matmul(X, self.rand_weight) + 1)
        X = self.dense(X)
        while tf.reduce_sum(tf.math.abs(X)) > 1:
            X /= 2
        return tf.reduce_sum(X)

In [23]:
net = FixedHiddenMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.52949965>

In [24]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(16, activation=tf.nn.relu)

    def call(self, inputs):
        return self.dense(self.net(inputs))

chimera = tf.keras.Sequential()
chimera.add(NestMLP())
chimera.add(tf.keras.layers.Dense(20))
chimera.add(FixedHiddenMLP())
chimera(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.97924674>

**5.2. Parameter Management**

In [25]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.01221426],
       [ 0.03614071]], dtype=float32)>

**5.2.1. Parameter Access**

In [26]:
print(net.layers[2].weights)

[<tf.Variable 'dense_13/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.6172626 ],
       [ 0.9529085 ],
       [-1.0036235 ],
       [ 0.69083595]], dtype=float32)>, <tf.Variable 'dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


5.2.1.1. Tartegeted Parameters


In [27]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


In [28]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

5.2.1.2. All Parameters at Once

In [29]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'dense_12/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.08216673,  0.20991951,  0.16389948,  0.06502217],
       [-0.4561521 , -0.16323745,  0.5127699 ,  0.20526892],
       [-0.76957875, -0.72368646, -0.2811209 , -0.24673659],
       [-0.0891636 , -0.58190536, -0.48662186,  0.34337276]],
      dtype=float32)>, <tf.Variable 'dense_12/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[ 0.08216673,  0.20991951,  0.16389948,  0.06502217],
       [-0.4561521 , -0.16323745,  0.5127699 ,  0.20526892],
       [-0.76957875, -0.72368646, -0.2811209 , -0.24673659],
       [-0.0891636 , -0.58190536, -0.48662186,  0.34337276]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[ 0.6172626 ],
       [ 0.9529085 ],
       [-1.0036235 ],
       [ 0.69083595]], dtype=float32), array([0.], dtype=float32)]


5.2.1.3. Collecting Parameters from Nested Blocks

In [30]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.],
       [0.]], dtype=float32)>

In [31]:
print(rgnet.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (2, 4)                    80        
                                                                 
 dense_18 (Dense)            (2, 1)                    5         
                                                                 
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'dense_15/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

**5.2.2. Parameter Initialization**

5.2.2.1. Built-in Initialization

In [33]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_19/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[ 0.00842769, -0.00487158, -0.00375726,  0.01544193],
        [ 0.00643937, -0.0024205 , -0.00691276,  0.00558222],
        [-0.00368167,  0.00996656, -0.00246682, -0.00941941],
        [ 0.00596205, -0.01282374,  0.01209693, -0.00343677]],
       dtype=float32)>,
 <tf.Variable 'dense_19/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [34]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_21/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_21/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [35]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(42)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'dense_23/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.16125482, -0.7053024 ,  0.4400174 ,  0.41602045],
       [ 0.198412  , -0.10393202, -0.2703187 ,  0.22289413],
       [-0.02388853,  0.11120248, -0.4162055 ,  0.1771577 ],
       [-0.84077126,  0.6988781 , -0.61582947,  0.75648624]],
      dtype=float32)>
<tf.Variable 'dense_24/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[42.],
       [42.],
       [42.],
       [42.]], dtype=float32)>


5.2.2.2. Custom Initialization

In [36]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_25/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.       , -0.       , -9.431968 ,  0.       ],
       [-0.       ,  9.424004 ,  5.081604 ,  0.       ],
       [ 9.029831 ,  0.       , -5.8811593,  0.       ],
       [ 9.44429  , -0.       , -8.492281 , -0.       ]], dtype=float32)>


In [37]:
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'dense_25/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.       ,  1.       , -8.431968 ,  1.       ],
       [ 1.       , 10.424004 ,  6.081604 ,  1.       ],
       [10.029831 ,  1.       , -4.8811593,  1.       ],
       [10.44429  ,  1.       , -7.492281 ,  1.       ]], dtype=float32)>

**5.2.3. Tied Parameters**

In [38]:
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)

print(len(net.layers) == 3)

True


**5.3. Deferred Initialization**

**5.3.1. Instantiating a Network**

In [39]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [40]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

In [41]:
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

**5.4. Custom Layers**

**5.4.1. Layers Without Parameters**

In [42]:
class CenteredLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

In [43]:
layer = CenteredLayer()
layer(tf.constant([1, 2, 3, 4, 5]))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2], dtype=int32)>

In [44]:
net = tf.keras.Sequential([tf.keras.layers.Dense(128), CenteredLayer()])

In [45]:
Y = net(tf.random.uniform((4, 8)))
tf.reduce_mean(Y)

<tf.Tensor: shape=(), dtype=float32, numpy=-9.313226e-10>

**5.4.2. Layers with Parameters**

In [46]:
class MyDense(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias
        return tf.nn.relu(linear)

In [47]:
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[ 0.03934978,  0.06715582,  0.03669367],
        [-0.0501098 , -0.00296197, -0.01195103],
        [ 0.03285902,  0.07854348, -0.01183984],
        [ 0.04249518,  0.00590346, -0.01924259],
        [ 0.0473627 ,  0.04631972,  0.06065185]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [48]:
dense(tf.random.uniform((2, 5)))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.07252261, 0.1289735 , 0.03117339],
       [0.07326901, 0.11683026, 0.01727588]], dtype=float32)>

In [49]:
net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net(tf.random.uniform((2, 64)))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.        ],
       [0.01479256]], dtype=float32)>

**5.5. File I/O**

**5.5.1. Loading and Saving Tensors**

In [50]:
x = tf.range(4)
np.save('x-file.npy', x)

In [51]:
x2 = np.load('x-file.npy', allow_pickle=True)
x2

array([0, 1, 2, 3], dtype=int32)

In [52]:
y = tf.zeros(4)
np.save('xy-files.npy', [x, y])
x2, y2 = np.load('xy-files.npy', allow_pickle=True)
(x2, y2)

(array([0., 1., 2., 3.]), array([0., 0., 0., 0.]))

In [53]:
mydict = {'x': x, 'y': y}
np.save('mydict.npy', mydict)
mydict2 = np.load('mydict.npy', allow_pickle=True)
mydict2

array({'x': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>, 'y': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>},
      dtype=object)

**5.5.2. Loading and Saving Model Parameters**

In [54]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

net = MLP()
X = tf.random.uniform((2, 20))
Y = net(X)

In [55]:
net.save_weights('mlp.params')

In [56]:
clone = MLP()
clone.load_weights('mlp.params')

In [57]:
Y_clone = clone(X)
Y_clone == Y

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

**5.6. GPUs**

**5.6.1. Computing Devices**

In [58]:
tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f94809fcb90>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x7f9480a3b320>)

In [59]:
len(tf.config.experimental.list_physical_devices('GPU'))

0

In [60]:
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if len(tf.config.experimental.list_physical_devices('GPU')) >= i + 1:
        return tf.device(f'/GPU:{i}')
    return tf.device('/CPU:0')

def try_all_gpus():
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    num_gpus = len(tf.config.experimental.list_physical_devices('GPU'))
    devices = [tf.device(f'/GPU:{i}') for i in range(num_gpus)]
    return devices if devices else [tf.device('/CPU:0')]

try_gpu(), try_gpu(10), try_all_gpus()

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f9480aa99b0>,
 [<tensorflow.python.eager.context._EagerDeviceContext at 0x7f9480b77640>])

**5.6.2. Tensors and GPUs**

In [61]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:CPU:0'

5.6.2.1. Storage on the GPU

In [62]:
with try_gpu():
    X = tf.ones((2, 3))
X

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [63]:
with try_gpu(1):
    Y = tf.random.uniform((2, 3))
Y

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.83010995, 0.98106194, 0.6100161 ],
       [0.94672143, 0.672706  , 0.5194926 ]], dtype=float32)>

5.6.2.2. Copying

In [64]:
with try_gpu(1):
    Z = X
print(X)
print(Z)

tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)


In [65]:
Y + Z

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.83011  , 1.9810619, 1.6100161],
       [1.9467214, 1.672706 , 1.5194926]], dtype=float32)>

In [66]:
with try_gpu(1):
    Z2 = Z
Z2 is Z

True

**5.6.3. Neural Networks and GPUs**

In [67]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    net = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1)])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [68]:
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-3.5117888],
       [-3.5117888]], dtype=float32)>

In [69]:
net.layers[0].weights[0].device, net.layers[0].weights[1].device

('/job:localhost/replica:0/task:0/device:CPU:0',
 '/job:localhost/replica:0/task:0/device:CPU:0')